In [1]:
import tarfile
import urllib.request
url = "http://pcai056.informatik.uni-leipzig.de/downloads/corpora/deu_wikipedia_2021_10K.tar.gz"
with urllib.request.urlopen(url) as f:
    stream = urllib.request.urlopen(url)
    f = tarfile.open(fileobj=stream, mode="r|gz")
    f.extractall()


In [2]:
import random
with open("deu_wikipedia_2021_10K/deu_wikipedia_2021_10K-sentences.txt", "r") as f:
    lines = f.read().split("\n")[:-1]
    sents = [l.split("\t")[1] for l in lines]
    random.shuffle(sents)
sents[:5]

['Die Enkel Benedict und Lucas Steiner führen das Architekturbüro von Joseph Steiner weiter.',
 'Ein älterer Bruder Rudi (geb. 1928) und eine jüngere Schwester Gerti (geb. 1940) wuchsen mit ihm in finanziell ärmlichen Verhältnissen, aber unter dem Eindruck politisch agierender Eltern auf.',
 'Bei lackiertem Holz sowie lackierten Vasen oder Pokalen kann die Lackierung abplatzen.',
 'Die Startmasse war auf 16,259 Tonnen festgelegt.',
 'Während der Ost-West-Konfrontation beteiligte sich die italienische Marine an verschiedenen anderen Einsätzen.']

In [3]:
import de_dep_news_trf
from itertools import chain
nlp = de_dep_news_trf.load()
def get_nouns(sent):
    doc = nlp(sent)
    return [w for w in doc if w.pos_ == "NOUN" and len(w.text) > 4]
nouns = list(chain(*[get_nouns(s) for s in sents[:100]]))


/opt/homebrew/Caskroom/miniconda/base/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [4]:
import re

def is_range(l):
    a = l[0]
    for b in l[1:]:
        if b - a != 1:
            return False
        a = b
    return True

def is_indefinite_nom(t):
    if t.morph.get("Case") == ["Nom"]:
        in_sing = any([c.lemma_ in ["ein", "einen"] for c in t.children])
        in_plur = not any(c.lemma_ in ["der"] for c in t.children)
        return in_sing or in_plur
    return False

def is_article(parent, t):
    return t.head == parent and t.lemma_ in ["der", "ein", "einen"]

def get_phrase(noun):
    if all([t.is_alpha for t in noun.subtree]) \
        and is_indefinite_nom(noun) \
        and is_range(sorted([t.i for t in noun.subtree])):
        ts = [t.text_with_ws for t in noun.subtree if not is_article(noun, t)]
        return re.sub(r"[^a-zäöÜA-ZÄÖÜ]+$", "", "".join(ts))
    return None

phrases = [phrase for noun in nouns if (phrase := get_phrase(noun)) is not None]

In [5]:
phrases[:10]

['Ganzjahreszeit',
 'Diese Hartkekse',
 'tellergroße Scheiben',
 'Moskauer Brücke',
 'Informatik oder Religionslehre',
 'Leistungskurs',
 'Deutsch oder eine Fremdsprache und Mathematik oder eine Naturwissenschaft',
 'Fremdsprache und Mathematik oder eine Naturwissenschaft',
 'Mathematik oder eine Naturwissenschaft',
 'Naturwissenschaft']

In [6]:
%load_ext dotenv
%dotenv
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def replace(a, b, sent):
    response = openai.Edit.create(
    engine="text-davinci-edit-001",
    input=sent,
    instruction=f"Ersetze \"{a}\" durch \"{b}\". Passe die Grammatik entsprechend an.",
    temperature=0,
    top_p=1
    )
    return response["choices"][0]["text"]

In [7]:
from functools import lru_cache

@lru_cache(maxsize=None)
def replace_batch(l):
    inputs = [f"({i})\nOriginal: {noun.doc.text}\nMit Ersetzung: ___\n" for i, (noun, _) in enumerate(l)]
    instructions = [f"({i}) Ersetze {noun.text} durch {phrase}. Passe die Grammatik entsprechend an.\n" for i, (noun, phrase) in enumerate(l)]
    response = openai.Edit.create(
    engine="text-davinci-edit-001",
    input="\n".join(inputs),
    instruction="\n".join(instructions),
    temperature=0,
    top_p=1
    )
    replaceds = re.findall(r"Mit Ersetzung: (.*)\n", response["choices"][0]["text"])
    return [dict(x=noun.doc.text,replace=noun.text,replace_with=phrase,y=replaced) for replaced, (noun, phrase) in zip(replaceds, l)]

In [8]:
import json
from tqdm.notebook import tqdm

random.shuffle(nouns)
random.shuffle(phrases)
data = []

def chunks(l, n):
    """
    Yield successive n-sized chunks from l.
    from https://stackoverflow.com/q/312443/10190810
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]

training_data = []
for chunk in tqdm(list(chunks(list(zip(nouns, phrases)), 5))):
    res = replace_batch(tuple(chunk))
    print(res)
    training_data = list(chain(training_data, res)) 
    with open("training_data.json", "w") as f:
        json.dump(training_data, f, indent=2, ensure_ascii=False)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/7 [00:00<?, ?it/s]

[{'x': 'Ein älterer Bruder Rudi (geb. 1928) und eine jüngere Schwester Gerti (geb. 1940) wuchsen mit ihm in finanziell ärmlichen Verhältnissen, aber unter dem Eindruck politisch agierender Eltern auf.', 'replace': 'Bruder', 'replace_with': 'Diese Hartkekse', 'y': 'Ein älterer Diese Hartkekse (geb. 1928) und eine jüngere Schwester Gerti (geb. 1940) wuchsen mit ihm in finanziell ärmlichen Verhältnissen, aber unter dem Eindruck politisch agierender Eltern auf.'}, {'x': 'Der botanische Garten Paradiso lädt zu ausgedehnten Spaziergängen ein, die besonders im Frühjahr reizvoll sind und die von dem hohen Felsen einen schönen Blick über die Stadt bieten.', 'replace': 'Blick', 'replace_with': 'Mathematik oder eine Naturwissenschaft', 'y': 'Der botanische Garten Paradiso lädt zu ausgedehnten Spaziergängen ein, die besonders im Frühjahr reizvoll sind und die von dem hohen Felsen einen schönen Mathematik oder eine Naturwissenschaft bieten.'}, {'x': 'Ein älterer Bruder Rudi (geb. 1928) und eine jün

In [9]:
len(training_data)

33